# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=10,kernel_size=5,stride=1,padding=2)
        self.conv2=nn.Conv2d(in_channels=10,out_channels=20,kernel_size=5,stride=1,padding=2)
        self.dropout=nn.Dropout2d(p=0.2)
        self.fc1=nn.Linear(in_features=980,out_features=50)
        self.fc2=nn.Linear(in_features=50,out_features=10)
        self.max_pool=nn.functional.max_pool2d
        self.relu=nn.functional.relu
    
    def forward(self,x):
        #input is 28*28*1
        #conv1(kernel=5,padding=2,filters=10) 28*28*1->28*28*10
        #max_pool(kernel=2) 28*28*10->14*14*10
        x=self.relu(self.max_pool(self.conv1(x),2))
        
        #conv2(kernel=5,padding=2,filters=10) 14*14*10->14*14*20
        #max_pool(kernel=2) 14*14*20->7*7*20
        x = self.relu(self.max_pool(self.dropout(self.conv2(x)),2))
        
        #flatten 7*7*20=980
        x=x.view(-1,980)
        
        #980->50
        x=self.relu(self.fc1(x))
        
        #50->10
        x=self.relu(self.fc2(x))
        
        return nn.functional.log_softmax(x,dim=1)
    
model = SimpleNet()
param=list(model.parameters())

# TODO:define loss function and optimiter
criterion = nn.functional.nll_loss
optimizer = torch.optim.SGD(param,lr=0.01,momentum=0.9)
device=torch.device('cpu')

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images,labels=images.to(device),labels.to(device)
        
        optimizer.zero_grad()
        criterion(model(images),labels).backward()
        optimizer.step()
        
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
correct=0
for images,labels in tqdm(train_loader):
    images,labels=images.to(device),labels.to(device)
    
    output=model(images)
    pred=output.max(1,keepdim=True)[1] #the second element of the returned tuple is the max_index
    correct+=pred.eq(labels.view_as(pred)).sum().item() # item returns the value of this tensor as a standard Python number. This only works for tensors with one element.
train_accuracy=100.*correct/len(train_loader.dataset)

correct=0
for images,labels in tqdm(test_loader):
    images,labels=images.to(device),labels.to(device)
    
    output=model(images)
    pred=output.max(1,keepdim=True)[1]
    correct+=pred.eq(labels.view_as(pred)).sum().item()
test_accuracy=100.*correct/len(test_loader.dataset)
    
    
    
    


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:04<00:00, 19.59it/s]


#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training accuracy: %0.2f%%' % (train_accuracy))
print('Testing accuracy: %0.2f%%' % (test_accuracy))

Training accuracy: 98.70%
Testing accuracy: 98.41%
